<a href="https://colab.research.google.com/github/applejxd/colaboratory/blob/master/algorithm/NonLinear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 例題

3次方程式
\begin{equation}
x^3+6x^2+21x+32=0
\end{equation}
の解は
\begin{equation}
x=-9^{1/3}+3^{1/3}-2,\ -9^{1/3}e^{i2\pi/3}+3^{1/3}e^{i4\pi/3}-2,\ -9^{1/3}e^{i4\pi/3}+3^{1/3}e^{i2\pi/3}-2
\end{equation}
または
\begin{equation}
x\sim-2.64,\ -1.68-3.05i,\ -1.68+3.05i
\end{equation}

In [9]:
def non_linear_func(x: float) -> float:
    return x**3 + 6*x**2 + 21*x + 32

def derivative_func(x: float) -> float:
    return 3*x**2 + 12*x + 21

x_real = -2.637834253

## 2分法



1. 解が含まれる領域 $[a,b]$ を事前に指定する
2. 中点を利用して領域を $[a,(a+b)/2]$ と $[(a+b)/2,b]$ の2つに分割する
3. 解を含む領域を中間値の定理で判定する
4. 解を含む領域を選択して新たに計算する領域として繰り返す
5. 領域幅が一定以下になったら終了

中間値の定理から領域 $[a,b]$ に対して $f(a)f(b)<0$ なら、この領域 $[a,b]$ に $f(x)=0$ の解が含まれる事がわかる。

In [10]:
from typing import Callable

def bisection(func: Callable[[float],float],
              left: float, right: float) -> float:
    '''
    二分法のアルゴリズム
    '''
    # 二分法で解けない場合
    if func(left) * func(right) >= 0:
        raise Exception('Cannot be solved by bisection method!')

    # 代数式の数値精度
    delta: float = 1e-10
    # 解の数値精度
    epsilon: float = 5e-6

    middle: float = 0.
    count: int = 0
    while True:
        # 中点計算
        middle = (left + right)/2.
        count += 1

        # 中間値の定理から解を含む区間を選択
        if func(left) * func(middle) < 0:
            right = middle
        else:
            left = middle

        # 終了判定
        if (abs(func(middle)) < delta):
            print(f"finished: f(x)={abs(func(middle)):.6e} (count={count})")
            break
        if (abs(left - right) < epsilon):
            print(f"finished: |left - right|={abs(left - right):.6e} (count={count})")
            break
    
    middle = (left+right)/2.
    return middle

In [11]:
x = bisection(non_linear_func, -3.0, 0)
print(f"Δx = {x-x_real:.6e}, f(x)={non_linear_func(x):.6e}")

finished: |left - right|=2.861023e-06 (count=20)
Δx = 1.134561e-06, f(x)=1.159317e-05


## はさみうち法

はさみうち法では2分法のアレンジとして、交点を中点ではなく直線近似の解を用いる。
\begin{equation}
y(x)\equiv\frac{f(a)-f(b)}{a-b}(x-a)+f(a)
\end{equation}
とした場合の
\begin{equation}
y(c)=0 ⇔ c=-f(a)\frac{a-b}{f(a)-f(b)}+a=\frac{bf(a)-af(b)}{f(a)-f(b)}
\end{equation}

In [12]:
def squeeze(func: Callable[[float],float], 
            left: float, right: float) -> float:
    '''
    はさみうち法のアルゴリズム
    '''
    # 二分法で解けない場合
    if func(left) * func(right) >= 0:
        raise Exception('Cannot be solved by squeeze method!')

    # 代数式の数値精度
    delta: float = 1e-10
    # 解の数値精度
    epsilon: float = 5e-6

    middle: float = 0.
    count: int = 0
    while True:
        # 中点計算
        f_left, f_right = func(left), func(right)
        middle = (right*f_left - left*f_right)/(f_left - f_right)
        count += 1

        # 中間値の定理から解を含む区間を選択
        if func(left) * func(middle) < 0:
            right = middle
        else:
            left = middle

        # 終了判定
        if (abs(func(middle)) < delta):
            print(f"finished: f(x)={abs(func(middle)):.6e} (count={count})")
            break
        if (abs(left - right) < epsilon):
            print(f"finished: |left - right|={abs(left - right):.6e} (count={count})")
            break
    
    f_left, f_right = func(left), func(right)
    middle = (right*f_left - left*f_right)/(f_left - f_right)
    return middle

In [13]:
x = squeeze(non_linear_func, -3.0, 0)
print(f"Δx = {x-x_real:.6e}, f(x)={non_linear_func(x):.6e}")

finished: f(x)=6.559020e-11 (count=13)
Δx = 2.544929e-10, f(x)=-1.033840e-11


## ニュートン・ラフソン法

非線形関数の$x=a$周りのテーラー展開を用いて
\begin{equation}
f(x)=f(a)+f'(a)(x-a)+\mathcal{O}(\epsilon)=0
⇔x=a-\frac{f(a)}{f'(a)}+\mathcal{O}(\epsilon)
\end{equation}
と解を1次近似する。

In [14]:
def newton1d(func: Callable[[float], float],
             derivative: Callable[[float], float], seed: float) -> float:
    '''
    1D のニュートン・ラフソン法
    '''
    # 代数式の数値精度
    delta: float = 1e-10

    result: float = 0.
    count: int = 0
    while True:
        result = result - func(result) / derivative(result)
        count += 1

        # 終了判定
        if (abs(func(result)) < delta):
            print(f"finished: f(x)={abs(func(result)):.4e} (count={count})")
            break
    
    return result

In [15]:
x = newton1d(non_linear_func, derivative_func, 0)
print(f"Δx = {x-x_real:.6e}, f(x)={non_linear_func(x):.6e}")

finished: f(x)=3.5527e-15 (count=5)
Δx = 2.555036e-10, f(x)=3.552714e-15
